In [1]:
library(ISLR)
library(boot)
library(dplyr)
set.seed(5)
train=sample(10000,5000)

Warning message:
"package 'ISLR' was built under R version 3.3.2"Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



# Validation Set approach for Logistic Reg

In [25]:
logreg.fn=function(data,index,predictors=c("income","balance"),response=c("predictions")){
    f=as.formula(paste0("default~",paste0(predictors,collapse="+")))
    
    glm.fit=glm(f,data=Default,family=binomial,subset = index)
    glm.probs=predict(glm.fit,Default[-index,],type="response")
    glm.preds=rep("No",nrow(data)-length(index))
    glm.preds[glm.probs>0.5] ="Yes"
    
    if(response=="predictions")
        glm.preds
    else if(response=="model")
        glm.fit
        
}
    

vsErrors=data.frame(seed=integer(),predictors=character(),error=double(),stringsAsFactors = FALSE)

for(seed in seq(6,20,by=2)){
    train=sample(10000,5000)
    predictors=c("income","balance")
    glm.preds=logreg.fn(Default,train,predictors)
    vse=sum(glm.preds!=Default[-train,]$default)/5000
    vsErrors=vsErrors %>% rbind(list(seed=seed,predictors=paste0(predictors,collapse=","),error=vse))
}
vsErrors$predictors=as.character(vsErrors$predictors)

for(seed in seq(6,20,by=2)){
    train=sample(10000,5000)
    predictors=c("income","balance","student")
    glm.preds=logreg.fn(Default,train,predictors)
    vse=sum(glm.preds!=Default[-train,]$default)/5000
    vsErrors=vsErrors %>% rbind(list(seed=seed,predictors=paste0(predictors,collapse=","),error=vse))
}

tapply(vsErrors$error,vsErrors$predictors,summary)

$`income,balance`
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.02380 0.02475 0.02490 0.02550 0.02565 0.02840 

$`income,balance,student`
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.02420 0.02515 0.02730 0.02678 0.02810 0.02880 


Using the `summay()` and `glm()` functions, determine est. Std. Errors for `default~income+balance`

In [34]:
glm.fit=glm(default~income+balance,data=Default,family=binomial)
summary(glm.fit)


Call:
glm(formula = default ~ income + balance, family = binomial, 
    data = Default)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4725  -0.1444  -0.0574  -0.0211   3.7245  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.154e+01  4.348e-01 -26.545  < 2e-16 ***
income       2.081e-05  4.985e-06   4.174 2.99e-05 ***
balance      5.647e-03  2.274e-04  24.836  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2920.6  on 9999  degrees of freedom
Residual deviance: 1579.0  on 9997  degrees of freedom
AIC: 1585

Number of Fisher Scoring iterations: 8


Write a function, `boot.fn()`, that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

Use the `boot()` function together with your boot.fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.

In [37]:
boot.fn = function(data,index)
    coef(glm(default~income+balance,data=Default,subset = index,family = binomial))

    
# Create 1000 data sets by sampling from Default, apply the boot.fn on that and assess the variability of the results
boot(Default,boot.fn,1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Default, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
         original        bias     std. error
t1* -1.154047e+01 -1.819609e-02 4.416655e-01
t2*  2.080898e-05 -5.361627e-09 4.821955e-06
t3*  5.647103e-03  1.052143e-05 2.326739e-04